# Install the requirments for ENV setup

In [ ]:
!pip install pip 
!pip install torch==2.1 
!pip install transformers==4.34.1 
!pip install accelerate==0.23.0 
!pip install bitsandbytes==0.41.1 
!pip install llava-torch==1.1.1 
# !pip install gdown

# load the lib and modules required for test run of LLMs

In [3]:
import textwrap
from io import BytesIO

import requests
import torch
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
from llava.conversation import SeparatorStyle, conv_templates
from llava.mm_utils import (
    KeywordsStoppingCriteria,
    get_model_name_from_path,
    process_images,
    tokenizer_image_token,
)
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# disable_torch_init()
# MODEL = "microsoft/llava-med-v1.5-mistral-7b"
# model_name = get_model_name_from_path(MODEL)
# model_name
disable_torch_init()
MODEL = "/kaggle/input/llava-med-bitfitbase-epoch5/other/default/1/llava-med-bitfitbase-train3"
model_name = get_model_name_from_path(MODEL)
model_name

'llava-med-bitfitbase-train3'

# Load the test images for testing the model

In [5]:
tokenizer, model, image_processor, context_len = load_pretrained_model(
                                                    model_path=MODEL, 
                                                    model_base=None, 
                                                    model_name=model_name, 
                                                    load_8bit=False )

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Some weights of the model checkpoint at /kaggle/input/llava-med-bitfitbase-epoch5/other/default/1/llava-med-bitfitbase-train3 were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.13.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.8.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.lay

In [6]:
# Load the saved dataset
data = torch.load('/kaggle/input/peft-thyroid-dataset/augmented_dataset.pt', weights_only=False)

# Print sizes
print("Train images:", data['train_images'].shape)
print("Train labels:", len(data['train_labels']))
print("Train IDs:", len(data['train_ids']))  # list of strings

print("Val images:", data['val_images'].shape)
print("Val labels:", len(data['val_labels']))
print("Val IDs:", len(data['val_ids']))

print("Test images:", data['test_images'].shape)
print("Test labels:", len(data['test_labels']))
print("Test IDs:", len(data['test_ids']))

Train images: torch.Size([1035, 3, 360, 560])
Train labels: 1035
Train IDs: 1035
Val images: (45, 3, 360, 560)
Val labels: 45
Val IDs: 45
Test images: (45, 3, 360, 560)
Test labels: 45
Test IDs: 45


In [7]:
def process_image(image):
    args = {"image_aspect_ratio": "pad"}
    image_tensor = process_images([image], image_processor, args)
    return image_tensor.to(model.device, dtype=torch.float16)

In [8]:
def create_prompt(prompt: str):
    CONV_MODE = 'llava_v0'
    conv = conv_templates[CONV_MODE].copy()
    roles = conv.roles
    prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt
    conv.append_message(roles[0], prompt)
    conv.append_message(roles[1], None)
    return conv.get_prompt(), conv

In [9]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, matthews_corrcoef, confusion_matrix,
    classification_report
)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, average_precision_score

def get_report(gt_labels, predicted_labels, all_probs, report_name):
    print("Report:", report_name)
    true_labels = gt_labels
    pred_labels = predicted_labels
    # Optional: sort class labels explicitly
    classes = sorted(set(true_labels + pred_labels))  # [2, 3, 4]
    
    # Binarize true labels
    y_true_bin = label_binarize(gt_labels, classes=class_labels)
    y_prob = np.array(all_probs)
    
    # Compute metrics
    roc_auc = roc_auc_score(y_true_bin, y_prob, average='macro')
    pr_auc = average_precision_score(y_true_bin, y_prob, average='macro')
    
    # Compute standard multi-class metrics
    metrics = {
        "Accuracy": accuracy_score(true_labels, pred_labels),
        "Macro Precision": precision_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Macro Recall": recall_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Macro F1 Score": f1_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Weighted F1 Score": f1_score(true_labels, pred_labels, average='weighted', zero_division=0),
        "Balanced Accuracy": balanced_accuracy_score(true_labels, pred_labels),
        "Matthews Correlation Coefficient": matthews_corrcoef(true_labels, pred_labels),
        "AUC-ROC (macro)": roc_auc,
        "AUC-PR (macro": pr_auc
    }
    
    # Display metrics summary table
    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Score'])
    print("=== Multi-class Summary Metrics ===")
    print(metrics_df.round(4))
    
    # Optional: show full classification report
    print("\n=== Per-class Report ===")
    print(classification_report(true_labels, pred_labels, digits=4))
    print("\n========================")

In [10]:
import torch.nn.functional as F

def classify_image_by_logits(image: Image, class_labels: list[str], prompt: str):
    image_tensor = process_image(image)
    prompt, conv = create_prompt(prompt)
    
    input_ids = tokenizer_image_token(
        prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt"
    ).unsqueeze(0).to(model.device)

    with torch.inference_mode():
        outputs = model(
            input_ids=input_ids,
            images=image_tensor,
            return_dict=True
        )

    # Get the logits for the next token prediction (only last token in prompt)
    next_token_logits = outputs.logits[0, -1]  # shape: [vocab_size]

    # Compute logits for all class labels
    label_scores = []
    for label in class_labels:
        token_ids = tokenizer(label, add_special_tokens=False)["input_ids"]
        logit_sum = sum([next_token_logits[token_id].item() for token_id in token_ids])
        label_scores.append(logit_sum)

    # Convert logits to probabilities
    label_scores_tensor = torch.tensor(label_scores)
    softmax_probs = F.softmax(label_scores_tensor, dim=0).tolist()

    # Get predicted label
    best_index = int(torch.argmax(label_scores_tensor))
    predicted_label = class_labels[best_index]

    return predicted_label, softmax_probs

def map_label_to_tirads(label):
    if "2" in label: return "TI-RADS 2"
    elif "3" in label: return "TI-RADS 3"
    elif "4" in label: return "TI-RADS 4"
    elif "5" in label: return "TI-RADS 5"
    else: return "Unknown"
    
custom_prompt = "The TI-RADS classification of the thyroid ultrasound image is: "
class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]

print(len(data['test_labels']))
gt_labels = []
predicted_labels = []
all_probs = []
for i in range(len(data['test_labels'])):
    img_tensor = data['test_images'][i]  # shape: (3, 360, 560)

    gt_label = map_label_to_tirads(data['test_labels'][i])
    gt_labels.append(gt_label)

    pred, probs = classify_image_by_logits(img_tensor, class_labels, custom_prompt)
    predicted_labels.append(pred)

    all_probs.append(probs)
    print(f"PatientId: {data['test_ids'][i]}, Label:{gt_label}, Predicted: {pred}")

baseline_report = get_report(gt_labels, predicted_labels, all_probs, "BitFit Test Data Report")

45
PatientId: 259, Label:TI-RADS 3, Predicted: TI-RADS 2
PatientId: 18, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 91, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 79, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 311, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 32, Label:TI-RADS 2, Predicted: TI-RADS 3
PatientId: 198, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 176, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 83, Label:TI-RADS 5, Predicted: TI-RADS 5
PatientId: 346, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 204, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 20, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 129, Label:TI-RADS 3, Predicted: TI-RADS 2
PatientId: 110, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 22, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 105, Label:TI-RADS 3, Predicted: TI-RADS 4
PatientId: 53, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 12, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 269, Label:TI-RADS 4, P

In [ ]:
baseline_report = get_report(gt_labels, predicted_labels, all_probs, "BitFit Test Data Report")

In [12]:
custom_prompt = "The TI-RADS classification of the thyroid ultrasound image is: "
class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]

print(len(data['val_labels']))
gt_labels = []
predicted_labels = []
all_probs = []
for i in range(len(data['val_labels'])):
    img_tensor = data['val_images'][i]  # shape: (3, 360, 560)

    gt_label = map_label_to_tirads(data['val_labels'][i])
    gt_labels.append(gt_label)

    pred, probs = classify_image_by_logits(img_tensor, class_labels, custom_prompt)
    predicted_labels.append(pred)

    all_probs.append(probs)
    print(f"PatientId: {data['val_ids'][i]}, Label:{gt_label}, Predicted: {pred}")
baseline_report = get_report(gt_labels, predicted_labels, all_probs, "BitFit Val Data Report")

45
PatientId: 233, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 76, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 30, Label:TI-RADS 5, Predicted: TI-RADS 4
PatientId: 141, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 217, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 11, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 378, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 334, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 74, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 247, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 89, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 192, Label:TI-RADS 2, Predicted: TI-RADS 4
PatientId: 343, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 179, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 220, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 227, Label:TI-RADS 4, Predicted: TI-RADS 4
PatientId: 5, Label:TI-RADS 2, Predicted: TI-RADS 2
PatientId: 335, Label:TI-RADS 4, Predicted: TI-RADS 3
PatientId: 372, Label:TI-RADS 5,

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
